
<a href="https://github.com/emanhamed/Houses-dataset" target="_blank">
https://github.com/emanhamed/Houses-dataset
</a>
<br>
<a href="https://arxiv.org/pdf/1609.08399.pdf" target="_blank">
https://arxiv.org/pdf/1609.08399.pdf
</a>
    
</p>

In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

In [2]:
cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv("D:/college-lessons/MA/FIU/AI-workshop/dataset/Houses-dataset-master/Houses Dataset/HousesInfo.txt", sep=" ", header=None, names=cols)
df

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226
...,...,...,...,...,...
530,5,2.0,2066,94531,399900
531,4,3.5,9536,94531,460000
532,3,2.0,2014,94531,407000
533,4,3.0,2312,94531,419000


In [3]:
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


In [4]:
zipcodes, counts = np.unique(df["zipcode"], return_counts=True)
#dict(zip(zipcodes, counts))

In [5]:
df.shape

(535, 5)

In [6]:
# loop over each of the unique zip codes and their corresponding
# count
for (zipcode, count) in zip(zipcodes, counts):
    # the zip code counts for our housing dataset is *extremely*
    # unbalanced (some only having 1 or 2 houses per zip code)
    # so let's sanitize our data by removing any houses with less
    # than 25 houses per zip code
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)

In [7]:
df.shape

(362, 5)

In [8]:
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
print(train.shape)
print(test.shape)

(271, 5)
(91, 5)


In [9]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [10]:
# initialize the column names of the continuous data
continuous = ["bedrooms", "bathrooms", "area"]

# performin min-max scaling each continuous feature column to
# the range [0, 1]
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(train[continuous])
testContinuous = cs.transform(test[continuous])

In [11]:
# one-hot encode the zip code categorical data (by definition of
# one-hot encoing, all output features are now in the range [0, 1])
zipBinarizer = LabelBinarizer().fit(df["zipcode"])
trainCategorical = zipBinarizer.transform(train["zipcode"])
testCategorical = zipBinarizer.transform(test["zipcode"])

In [12]:
trainCategorical.shape

(271, 7)

In [13]:
# construct our training and testing data points by concatenating
# the categorical features with the continuous features
trainX = np.hstack([trainCategorical, trainContinuous])
testX = np.hstack([testCategorical, testContinuous])

print(trainX.shape)
print(testX.shape)

(271, 10)
(91, 10)


In [14]:
dim = trainX.shape[1]
# define our MLP network
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

In [15]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [16]:
model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=200, batch_size=8)

Epoch 1/200
34/34 [==============================] - 2s 32ms/step - loss: 454.3910 - val_loss: 154.8064
Epoch 2/200
34/34 [==============================] - 0s 5ms/step - loss: 126.0024 - val_loss: 91.6062
Epoch 3/200
34/34 [==============================] - 0s 5ms/step - loss: 76.4974 - val_loss: 83.6727
Epoch 4/200
34/34 [==============================] - 0s 5ms/step - loss: 75.0003 - val_loss: 76.7475
Epoch 5/200
34/34 [==============================] - 0s 5ms/step - loss: 68.2477 - val_loss: 70.8461
Epoch 6/200
34/34 [==============================] - 0s 5ms/step - loss: 72.6013 - val_loss: 64.5825
Epoch 7/200
34/34 [==============================] - 0s 5ms/step - loss: 58.2849 - val_loss: 59.4816
Epoch 8/200
34/34 [==============================] - 0s 5ms/step - loss: 51.0797 - val_loss: 59.2486
Epoch 9/200
34/34 [==============================] - 0s 5ms/step - loss: 49.1509 - val_loss: 51.9132
Epoch 10/200
34/34 [==============================] - 0s 5ms/step - loss: 53.1872 - val

34/34 [==============================] - 0s 5ms/step - loss: 21.6638 - val_loss: 23.4061
Epoch 161/200
34/34 [==============================] - 0s 5ms/step - loss: 24.3166 - val_loss: 22.6870
Epoch 162/200
34/34 [==============================] - 0s 5ms/step - loss: 19.8786 - val_loss: 23.5650
Epoch 163/200
34/34 [==============================] - 0s 5ms/step - loss: 19.3214 - val_loss: 24.1160
Epoch 164/200
34/34 [==============================] - 0s 5ms/step - loss: 20.8507 - val_loss: 22.5554
Epoch 165/200
34/34 [==============================] - 0s 5ms/step - loss: 19.3159 - val_loss: 24.6889
Epoch 166/200
34/34 [==============================] - 0s 5ms/step - loss: 21.6185 - val_loss: 24.6432
Epoch 167/200
34/34 [==============================] - 0s 5ms/step - loss: 22.3426 - val_loss: 26.1673
Epoch 168/200
34/34 [==============================] - 0s 5ms/step - loss: 21.3524 - val_loss: 23.5960
Epoch 169/200
34/34 [==============================] - 0s 5ms/step - loss: 19.5319 - va

In [17]:
preds = model.predict(testX)
preds.shape
testY.shape
preds.flatten().shape

(91,)

In [18]:
# make predictions on the testing data
preds = model.predict(testX)
 
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
 
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

avg. house price: $533,388.27, std house price: $493,403.08
mean: 23.78%, std: 23.78%


## reference 

https://www.pyimagesearch.com/2019/01/21/regression-with-keras/

## Contact me

### LinkedIn Account
https://www.linkedin.com/in/mohammed-behjoo-446098b7/


### YouTube Channel
https://www.youtube.com/channel/UCudI0gvPh-YbiY2zLM7im-w


### GitHub
https://github.com/mohammedbehjoo